In [ ]:
import requests
import datetime

year = datetime.date.today().year

url = f"https://feriadosargentina.com.ar/feriados/{year}"
response = requests.get(url)
print(response)

In [ ]:
response.text

In [ ]:
with open("response.txt", "w") as f:
  f.write(response.text)

In [ ]:
import re

pattern = r"<td id='([^']+)' style='([^']+)'>"
result = re.findall(pattern, response.text)
list(result)

In [ ]:
import pandas as pd

df = pd.DataFrame(result, columns=["raw_date", "style"])
df

In [ ]:
def get_tipo(style):
  pattern = f"<td style='{style} width: 50px;'></td>\s*<td class='text-center'>([^<]+)</td>"
  result = re.findall(pattern, response.text)
  return result[0] if len(result) != 0 else None

df["tipo"] = df["style"].apply(get_tipo)

In [ ]:
a_eliminar = df[df["tipo"].isna()]
print(a_eliminar)
df.drop(index=a_eliminar.index, inplace=True)

In [ ]:
import calendar

list(calendar.month_name)

In [ ]:
from datetime import date

MONTHS = list(calendar.month_name)

def get_date(raw_date):
  pattern = "([^-]+)-([^-]+)-(\d+)"
  result = re.findall(pattern, raw_date)
  raw_month, raw_day, raw_year = result[0]
  month = MONTHS.index(raw_month)
  return date(int(raw_year), month, int(raw_day))

df["fecha"] = df["raw_date"].apply(get_date)


In [ ]:
MESES = ["", "enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]

def get_name(date):
  mes = MESES[date.month]
  pattern = f"<tr>\s*<td><a[^>]*>([^<]+)</a></td>\s*<td> <a[^>]*>[^\d]+{date.day:02d} de {mes} {date.year}"
  result = re.findall(pattern, response.text)
  if len(result) == 0:
    pattern = f"<tr>\s*<td>([^<]+)</td>\s*<td>[^\d]+{date.day:02d} de {mes} {date.year}"
    result = re.findall(pattern, response.text)
  if len(result) == 0:
    print(date)
    return None
  return result[0]

df["motivo"] = df["fecha"].apply(get_name)

In [ ]:
df[["fecha", "tipo", "motivo"]].to_json(f"feriados_{year}.json"
,orient="records"
,date_format="iso"
,force_ascii=False
,indent=2
)

In [ ]:
df["tipo"]